# LATAM Data Science Challenge

## Problema 

El problema consiste en predecir la probabilidad de atraso de los vuelos que 
aterrizan o despegan del aeropuerto de Santiago de Chile (SCL). Para eso les 
entregamos un dataset usando datos públicos y reales donde cada fila 
corresponde a un vuelo que aterrizó o despegó de SCL. Para cada vuelo se cuenta
con la siguiente información:** 


In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from scipy import stats

import statsmodels.api as sm
from IPython.display import display
from pandas_profiling import ProfileReport
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

## **Features**:

**Fecha-I :** Fecha y hora programada del vuelo. 

**Vlo-I :** Número de vuelo programado. 

**Ori-I :** Código de ciudad de origen programado. 

**Des-I :** Código de ciudad de destino programado. 

**Emp-I :** Código aerolínea de vuelo programado. 

**Fecha-O :** Fecha y hora de operación del vuelo. 

**Vlo-O :** Número de vuelo de operación del vuelo. 

**Ori-O :** Código de ciudad de origen de operación 

**Des-O :** Código de ciudad de destino de operación. 

**Emp-O :** Código aerolínea de vuelo operado. 

**DIA :** Día del mes de operación del vuelo. 

**MES :** Número de mes de operación del vuelo. 

**AÑO :** Año de operación del vuelo. 

**DIANOM :** Día de la semana de operación del vuelo. 

**TIPOVUELO :** Tipo de vuelo, I =Internacional, N =Nacional. 

**OPERA :** Nombre de aerolínea que opera. 

**SIGLAORI :** Nombre ciudad origen. 

**SIGLADES :** Nombre ciudad destino.

### --- Synthetics+ ---
**Hora-O :** Hora de operación de vuelo

**Hora-I :** Hora programada de vuelo

**temporada_alta :** 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.

**dif_min :** diferencia en minutos entre Fecha-O y Fecha-I .

**atraso_15 :** 1 si dif_min > 15, 0 si no.

**periodo_dia :** mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), con base a
Fecha-I .

**Dummies :**
    
    ['Ori-I',
    
    'Des-I',
    
    'Emp-I',
    
    'Ori-O',
    
    'Des-O',
    
    'Emp-O',
    
    'DIANOM',
    
    'TIPOVUELO',
    
    'OPERA',
    
    'SIGLAORI',
    
    'SIGLADES']

Desafío 
1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto? 2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv : 
○ temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no. ○ dif_min : diferencia en minutos entre Fecha-O y Fecha-I . 
○ atraso_15 : 1 si dif_min > 15, 0 si no. 
○ periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I . 
3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo? ¿Qué variables esperarías que más influyeran en predecir atrasos? 
4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo. Siéntete libre de generar variables adicionales y/o complementar con variables externas. 
5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la performance? 


## Desafío 
### #1

### 1.1 ¿Cómo se distribuyen los datos? 

In [101]:
df = pd.read_csv('./data/dataset_SCL.csv')

In [102]:
df.describe()

,DIA,MES,AÑO
count,68206.000000,68206.000000,68206.000000
mean,15.714790,6.622585,2017.000029
std,8.782886,3.523321,0.005415
min,1.000000,1.000000,2017.000000
25%,8.000000,3.000000,2017.000000
50%,16.000000,7.000000,2017.000000
75%,23.000000,10.000000,2017.000000
max,31.000000,12.000000,2018.000000


In [103]:
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'])
df['Fecha-O'] = pd.to_datetime(df['Fecha-O'])

In [104]:
df['Hora-O'] = df['Fecha-O'].dt.hour
df['Hora-I'] = df['Fecha-I'].dt.hour

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha-I    68206 non-null  datetime64[ns]
 1   Vlo-I      68206 non-null  object        
 2   Ori-I      68206 non-null  object        
 3   Des-I      68206 non-null  object        
 4   Emp-I      68206 non-null  object        
 5   Fecha-O    68206 non-null  datetime64[ns]
 6   Vlo-O      68205 non-null  object        
 7   Ori-O      68206 non-null  object        
 8   Des-O      68206 non-null  object        
 9   Emp-O      68206 non-null  object        
 10  DIA        68206 non-null  int64         
 11  MES        68206 non-null  int64         
 12  AÑO        68206 non-null  int64         
 13  DIANOM     68206 non-null  object        
 14  TIPOVUELO  68206 non-null  object        
 15  OPERA      68206 non-null  object        
 16  SIGLAORI   68206 non-null  object       

In [107]:
df.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,Hora-O,Hora-I
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami,23,23
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami,23,23
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami,23,23
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami,23,23
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami,23,23
